In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
spark = SparkSession. \
builder. \
appName('UNIR'). \
config("spark.executor.memory", "1g"). \
config("spark.driver.memory", "2g"). \
config('spark.dirver.maxResultSize', '1g'). \
config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2'). \
getOrCreate()

In [2]:
def promediarValores(df):
   df.createOrReplaceTempView("resultadoMedio")
   promedios = spark.sql("""SELECT tipo, AVG(total) AS promedio FROM resultadoMedio GROUP BY tipo ORDER BY promedio DESC""")
   return promedios

In [3]:
tiposStreamingDF = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "127.0.0.1:9092")
  .option("subscribe", "promedios")
  .load())

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F
esquema = StructType([\
  StructField("tipo", StringType()),\
  StructField("total", DoubleType())\
])

In [5]:
parsedDF = tiposStreamingDF\
     .select("value")\
     .withColumn("value", F.col("value").cast(StringType()))\
     .withColumn("parejas", F.from_json(F.col("value"), esquema))\
     .withColumn("tipo", F.col("parejas.tipo"))\
     .withColumn("total", F.col("parejas.total"))

In [6]:
promediosStreamingDF = promediarValores(parsedDF)

In [8]:
salida = promediosStreamingDF\
                    .writeStream\
                    .queryName("AgregacionPromedios")\
                    .outputMode("complete")\
                    .format("memory")\
                    .option("checkpointLocation", "hdfs://localhost:9000/tmp/checkpoints/promedios")\
                    .start()

In [9]:
promediosDF = spark.sql("select * from AgregacionPromedios")

In [10]:
promediosDF.show()

+----+--------+
|tipo|promedio|
+----+--------+
+----+--------+



In [ ]:
kafka-topics.bat --list --bootstrap-server localhost:9092

kafka-topics.bat --create --topic promedios --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1

kafka-topics.bat --list --bootstrap-server localhost:9092

kafka-console-producer.bat --topic promedios --bootstrap-server localhost:9092

In [ ]:
{"tipo": "gasto", "total": 3.5}
{"tipo": "ingreso", "total": 7.0}
{"tipo": "ingreso", "total": 6.5}
{"tipo": "gasto", "total": 1.5}
{"tipo": "gasto", "total": 2.5}

In [13]:
promediosDF.show()

+-----+--------+
| tipo|promedio|
+-----+--------+
|gasto|     3.5|
+-----+--------+



In [14]:
promediosDF.show()

+-------+--------+
|   tipo|promedio|
+-------+--------+
|ingreso|     7.0|
|  gasto|     3.5|
+-------+--------+



In [15]:
promediosDF.show()

+-------+--------+
|   tipo|promedio|
+-------+--------+
|ingreso|    6.75|
|  gasto|     3.5|
+-------+--------+



In [ ]:
#kafka-topics.bat --delete --topic promedios --bootstrap-server localhost:9092